In [ ]:
!pip install ultralytics pillow opencv-python --quiet

from ultralytics import YOLO
from PIL import Image
import cv2
import numpy as np
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import files

uploaded = files.upload()
image_path = list(uploaded.keys())[0]
print("Используемое изображение:", image_path)

Saving 5249203991911009462.jpg to 5249203991911009462 (1).jpg
Используемое изображение: 5249203991911009462 (1).jpg


In [ ]:
model = YOLO("yolov8n.pt")
model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [ ]:
results = model.predict(
    source=image_path,
    save=True
)

res = results[0]

print("Готово, детекция выполнена.")


image 1/1 /content/5249203991911009462 (1).jpg: 640x480 1 person, 6 cars, 335.3ms
Speed: 20.6ms preprocess, 335.3ms inference, 34.4ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/runs/detect/predict
Готово, детекция выполнена.


In [ ]:
boxes = res.boxes
num_objects = len(boxes)

print("Количество обнаруженных объектов:", num_objects)

if num_objects > 0:
    cls_id = int(boxes.cls[0].item())
    class_name = model.names[cls_id]
    xyxy = boxes.xyxy[0].tolist()
    conf = float(boxes.conf[0].item())

    print("\nПервый объект:")
    print("ID класса:", cls_id)
    print("Класс:", class_name)
    print("Координаты (xyxy):", xyxy)
    print("Confidence:", conf)
else:
    print("Объекты не найдены.")

Количество обнаруженных объектов: 7

Первый объект:
ID класса: 2
Класс: car
Координаты (xyxy): [171.393798828125, 764.716552734375, 668.8277587890625, 1009.98876953125]
Confidence: 0.9353635907173157


In [ ]:
#Низкий порог уверенности
results_low_conf = model.predict(
    source=image_path,
    conf=0.25,
    save=True,
    project="runs/conf_test",
    name="conf_0_25"
)

num_low = len(results_low_conf[0].boxes)
print("conf=0.25, количество объектов:", num_low)

#Высокий порог уверенности
results_high_conf = model.predict(
    source=image_path,
    conf=0.75,
    save=True,
    project="runs/conf_test",
    name="conf_0_75"
)

num_high = len(results_high_conf[0].boxes)
print("conf=0.75, количество объектов:", num_high)


image 1/1 /content/5249203991911009462 (1).jpg: 640x480 1 person, 6 cars, 110.1ms
Speed: 1.6ms preprocess, 110.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/runs/conf_test/conf_0_25
conf=0.25, количество объектов: 7

image 1/1 /content/5249203991911009462 (1).jpg: 640x480 2 cars, 123.7ms
Speed: 1.5ms preprocess, 123.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/runs/conf_test/conf_0_75
conf=0.75, количество объектов: 2


In [ ]:
#Строгий NMS - удаляет почти все перекрывающиеся рамки
results_iou_low = model.predict(
    source=image_path,
    conf=0.25,
    iou=0.2,
    save=True,
    project="runs/iou_test",
    name="iou_0_2"
)

num_iou_low = len(results_iou_low[0].boxes)
print("iou=0.2, количество рамок:", num_iou_low)

#Мягкий NMS - оставляет даже перекрывающиеся рамки
results_iou_high = model.predict(
    source=image_path,
    conf=0.25,
    iou=0.8,
    save=True,
    project="runs/iou_test",
    name="iou_0_8"
)

num_iou_high = len(results_iou_high[0].boxes)
print("iou=0.8, количество рамок:", num_iou_high)


image 1/1 /content/5249203991911009462 (1).jpg: 640x480 1 person, 5 cars, 123.2ms
Speed: 2.0ms preprocess, 123.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/runs/iou_test/iou_0_2
iou=0.2, количество рамок: 6

image 1/1 /content/5249203991911009462 (1).jpg: 640x480 1 person, 7 cars, 118.0ms
Speed: 1.6ms preprocess, 118.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/runs/iou_test/iou_0_8
iou=0.8, количество рамок: 8


In [ ]:
# Фильтрация по двум классам: person(0) и car(2)
results_classes = model.predict(
    source=image_path,
    conf=0.25,
    classes=[0, 2],
    save=True,
    project="runs/classes_test",
    name="person_and_car"
)

res_c = results_classes[0]
boxes_c = res_c.boxes

print("Количество объектов после фильтрации:", len(boxes_c))
for i in range(len(boxes_c)):
    class_id = int(boxes_c.cls[i].item())
    class_name = model.names[class_id]
    conf = float(boxes_c.conf[i].item())
    print(f"{i}: Класс = {class_name}, conf = {conf:.3f}")


image 1/1 /content/5249203991911009462 (1).jpg: 640x480 1 person, 7 cars, 135.5ms
Speed: 3.1ms preprocess, 135.5ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/runs/classes_test/person_and_car
Количество объектов после фильтрации: 8
0: Класс = car, conf = 0.935
1: Класс = car, conf = 0.809
2: Класс = person, conf = 0.642
3: Класс = car, conf = 0.639
4: Класс = car, conf = 0.550
5: Класс = car, conf = 0.504
6: Класс = car, conf = 0.339
7: Класс = car, conf = 0.300


In [ ]:
uploaded_video = files.upload()
video_path = list(uploaded_video.keys())[0]
print("Видео загружено:", video_path)

Saving video5249203991451041699.mp4 to video5249203991451041699 (1).mp4
Видео загружено: video5249203991451041699 (1).mp4


In [ ]:
#Детекция на видео
video_results = model.predict(
    source=video_path,
    save=True,
    project="runs/video_test",
    name="video_detect",
    stream=True
)

inference_times = []
frame_count = 0

for res in video_results:
    frame_count += 1
    inference_ms = res.speed['inference']
    inference_times.append(inference_ms)

avg_inference_ms = sum(inference_times) / len(inference_times)
fps = 1000.0 / avg_inference_ms

print(f"Кадров обработано: {frame_count}")
print(f"Среднее время инференса: {avg_inference_ms:.2f} ms")
print(f"Средний FPS: {fps:.2f}")



video 1/1 (frame 1/153) /content/video5249203991451041699 (1).mp4: 640x352 1 car, 100.1ms
video 1/1 (frame 2/153) /content/video5249203991451041699 (1).mp4: 640x352 (no detections), 94.2ms
video 1/1 (frame 3/153) /content/video5249203991451041699 (1).mp4: 640x352 (no detections), 95.8ms
video 1/1 (frame 4/153) /content/video5249203991451041699 (1).mp4: 640x352 (no detections), 89.9ms
video 1/1 (frame 5/153) /content/video5249203991451041699 (1).mp4: 640x352 (no detections), 105.7ms
video 1/1 (frame 6/153) /content/video5249203991451041699 (1).mp4: 640x352 (no detections), 87.9ms
video 1/1 (frame 7/153) /content/video5249203991451041699 (1).mp4: 640x352 (no detections), 88.2ms
video 1/1 (frame 8/153) /content/video5249203991451041699 (1).mp4: 640x352 (no detections), 87.5ms
video 1/1 (frame 9/153) /content/video5249203991451041699 (1).mp4: 640x352 (no detections), 91.0ms
video 1/1 (frame 10/153) /content/video5249203991451041699 (1).mp4: 640x352 (no detections), 88.2ms
video 1/1 (frame

1) Почему инференс на видео медленнее, чем на статическом изображении?

Инференс на видео работает медленнее, потому что видео состоит из множества отдельных кадров, и модель обрабатывает каждый кадр как отдельную картинку. То есть вместо одной детекции - десятки или сотни. Плюс дополнительно тратится время на чтение и декодирование видео, обработку кадров и сохранение результата. Поэтому итоговая скорость заметно ниже.

2) В чём главное отличие между классификацией и детекцией объектов?

Классификация выдаёт один класс на всю картинку - то есть говорит что изображено.
Детекция объектов определяет не только что, но и где расположен каждый объект, выводя bounding box и уверенность для каждого из них. Поэтому детекция - более сложная задача, чем классификация.

3) Зачем используется метрика IOU (Intersection Over Union)?

IOU показывает, насколько сильно пересекаются две рамки.
Эта метрика нужна для: оценки качества детекции (насколько предсказанная рамка совпадает с настоящей), работы алгоритма NMS, который удаляет лишние дублирующиеся боксы. Чем выше IOU, тем больше рамки накладываются друг на друга.

4) Какова роль NMS (Non-Maximum Suppression) в YOLO?

NMS нужен для удаления лишних пересекающихся рамок.
YOLO часто выдает много кандидатов для одного и того же объекта.
NMS оставляет рамку с наибольшей уверенностью и удаляет остальные, если они слишком сильно пересекаются. Это помогает убрать дубликаты и делает результат аккуратным.

В ходе выполнения практической работы я познакомился с основными возможностями модели YOLOv8 для детекции объектов на изображениях и видеопотоке. Я последовательно выполнил детекцию на статическом изображении, изучил влияние порогов уверенности и параметра IOU на количество найденных объектов, а также провёл фильтрацию по классам. Эксперименты показали, что изменение параметров модели значительно влияет на результаты: при низком confidence модель определяет больше объектов, но возрастает количество ложных срабатываний, а при повышенном пороге остаются только наиболее уверенные детекции. Аналогично, настройка IOU влияет на то, сколько пересекающихся рамок будет оставлено после NMS.

Кроме работы с изображениями, я выполнил детекцию на видеоролике и проанализировал производительность модели. Инференс на видео оказался медленнее, чем на одиночном изображении, что связано с тем, что модель обрабатывает каждый кадр отдельно. Также я сравнил детекцию с задачей классификации и изучил назначение метрик IOU и NMS в конвейере детекции объектов. В результате работы я получил представление о том, как работает современная детекция объектов, как изменяются её результаты при разных настройках, и как применять модель YOLOv8 на практике для анализа изображений и видео.